In [3]:
from __future__ import unicode_literals
import math
import numpy as np

In [4]:
from zlib import crc32

def bytes_to_float(b):
    return float(crc32(b) & 0xffffffff) / 2**32
def h(s, encoding="utf-8"):
    return round(bytes_to_float(s.encode(encoding)),3)
dirr=['dirr1','dirr2','dirr3','dirr4','dirr5']
addr=['addr1','addr2','addr3']
h_dirr=[]
h_addr=[]
for d in dirr:
    h_dirr.append(h(d))
for a in addr:
    h_addr.append(h(a))
print(h_dirr,h_addr)

[0.965, 0.43, 0.098, 0.529, 0.939] [0.833, 0.298, 0.231]


In [5]:
def c(r,x):
    return np.dot(r,x)
    
# x=["cpu","mem","io","disk"]
x_fir=[0.2,0.1,0.3,0.2]

r=[0.116,0.368,0.258,0.258]
print(c(r,x_fir))

0.189


In [6]:
# for each service at t time
def res_util(r,x,p1,t):
    # not rewrite
    util=[]
    # rewite
    util_re=[]
    u=[]
    # t=0    
    for t in range(t+1):
        if (t==0):
            util.append(np.dot(r,x[0]))
            util_re.append(np.dot(r,x[0]))
        else:            
            util.append(np.dot(r,x[t]))
            util_re.append(p1*util[t]+(1-p1)*util_re[t-1])
        
    return util_re

# x.append([0.3,0.2,0.2,0.4])
# x.append([0.4,0.3,0.4,0.5])
x1=[[0.2,0.1,0.3,0.1],[0.3,0.4,0.2,0.2]]

x2=[[0.1]]
x3=[[0.2]]
x=[[[0.2,0.1,0.3,0.1],[0.3,0.4,0.2,0.2]],[[0.1,0.1,0.2,0.1],[0.3,0.2,0.2,0.1]],[[0.1,0.2,0.3,0.1],[0.3,0.4,0.3,0.2]]]
# u1=res_util(r,x[0],0.2,1)
# print(u1)
u=[]
print(len(x))
for i in range(len(x)):
        tmp=res_util(r,x[i],0.2,1)
        
        u.append(tmp)
#         u.append(res_util(r,x[i],0.2,0))
print(u)

3
[[0.1632, 0.18760000000000002], [0.12580000000000002, 0.13780000000000003], [0.1884, 0.21292000000000003]]


In [7]:
# for each service at t time
def stay_time(Q,t1,t2):
    stay_sum=[]
    stay_av=[]
    
    stay=0
    for i in range(len(t1)):
        stay=stay+t1[i]+t2[i]
    stay_sum.append(stay)
    stay_av.append(stay/(len(t1)))
    return stay_av

Q=['dirr1','dirr2','dirr3']
t1=[0.1,0.2,0.15]
t2=[0.1,0.2,0.15]
print(stay_time(Q,t1,t2))

[0.30000000000000004]


In [16]:
def load(r,x,p1,t,Q,t1,t2):
    load1=[]
    
    for i in range(len(x)): 
        tmp=[]
        for j in range(t+1):            
            tmp.append(u[i][j]+stay_time(Q,t1,t2))
       
        load1.append(tmp)
    return load1

def load_ave(l,t):
    load_sum=[]
    load_av=[]
    
    for j in range(t+1):
        tmp=0
        for i in range(len(l)):
            tmp=tmp+l[i][j]
        load_sum.append(tmp)
        load_av.append(tmp/len(h_addr))
    return load_sum,load_av
l=[]
l=load(r,x,0.2,1,Q,t1,t2)
load_sum,load_av=load_ave(l,1)
print(l)
print(load_sum,load_av)

[[array([0.4632]), array([0.4876])], [array([0.4258]), array([0.4378])], [array([0.4884]), array([0.51292])]]
[array([1.3774]), array([1.43832])] [array([0.45913333]), array([0.47944])]


In [33]:
def c_re(p2,t):
    #t=0,c1:c2:c3=1:2:2,c=[0.2,0.4,0.4]
    c=[[0.2,0.4,0.4]]  
    load_sum,load_av=load_ave(l,t)   
    for i in range(t+1):
        tmp = []
        for j in range(len(h_addr)):        
            if(i!=0):                
                tmp.append((1-p2)*c[i-1][j]+p2*(load_av[i]*c[i-1][j]/l[j][i]+c[0][j]))
        if(len(tmp)!=0):
                c.append(tmp)
    return c
c=c_re(0.3,1)
print(c)
print(c[1][0])

[[0.2, 0.4, 0.4], [array([0.2589959]), array([0.53141343]), array([0.5121672])]]
[0.2589959]


In [35]:
c=c_re(0.3,1)
print(c[0][1])
print(len(h_dirr))
# when t!=0
def dist(h_dirr,h_addr,r,x,t):
    d=[]
    for i in range(len(h_dirr)):
        tmp=[]
        for j in range(len(h_addr)):
            tmp.append((-1)*(math.log(1-abs(h_dirr[i]-h_addr[j])))/c[t][j][0])
        d.append(tmp) 
    return d
dist_1=dist(h_dirr,h_addr,r,x,1)
print(dist_1)

0.4
5
[[0.5465861245811369, 2.0692228033573694, 2.585598943723082], [1.9916846908412726, 0.2663906407319332, 0.43324588552914534], [5.127592606019398, 0.4199057426575611, 0.27865178075301716], [1.3992716527878455, 0.4942748795149199, 0.6908327500433905], [0.4326304182959093, 1.9277512218129815, 2.4035148634342587]]


In [28]:
def BBLA(dist):
    s=[]
    d_min=[]
    for d in dist:
        d_min.append(min(d))
        s.append(d.index(min(d)))
    return s,d_min
s,d=BBLA(dist)
print(s)
print(d)

[0, 1, 2, 1, 0]
[0.5465861245811369, 0.2663906407319332, 0.27865178075301716, 0.4942748795149199, 0.4326304182959093]


In [1]:
# def BBLA(h_dirr,h_addr,x,t):
#     s_fin=[]
#     nearest=dist(h_dirr[0],h_addr[0],r,x,t)
#     for j in range(len(h_dirr)):
#         for i in range(len(h_addr)):
#             if (dist(h_dirr[i],h_addr[i],r,x,t)<nearest):
#                 nearest=dist(h_dirr[i],h_addr[i],r,x,t)
#                 s=S[i]
#         s_fin.append(s)
#     return s,nearest
# # t=0


In [34]:
# n:the number of services
# c[[c1,c2,...ct],[c1,c2,...,ct],,,[c1,c2,...ct]]:compute of each service at different time
def reward(t,time_p):
    a=0
    b=0
    r=0
    for k in range(t+1):#at different time
        a=a+time_p**k
        for i in range(len(h_addr)):#in different services
            b=b+abs(c[k][i]-(l[i][k]/load_sum[k]))
        r=r+a*b
        return r
r=reward(1,0.4)
print(r)

[0.27257151]


In [ ]:
def f(t,p1,p2):
    for m in range(t):
        s,nearest=BBLA(dist)
        f=f+np.log(nearest)
    f=f*reward
    return f

In [ ]:
def gradient_descent(p1,p2,rate):
    delta=0.00000001
    derivate_p1=(f(p1+delta,p2)-f(p1,p2))/delta
    derivate_p2=(f(p1,p2+delta)-f(p1,p2))/delta
#     rate=0.02
    p1=p1-rate*derivate_p1
    p2=p2-rate*derivate_p2
    return(p1,p2)

In [ ]:
p1=1.0
p2=1.0
for i in range(1000):
    p1,p2=gradient_descent(p1,p2)
    print('p = {:6f},{:6f}, f(x) = {:6f}'.format(p1,p2,f(x)))